In [1]:
import findspark
findspark.init()

import pandas as pd
from tqdm import tqdm
import csv
import random
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

random.seed(1999)

letters = string.ascii_lowercase
letters_upper = string.ascii_uppercase
for _i in range(0, 10):
    letters += letters

for _i in range(0, 10):
    letters += letters_upper


def random_string(stringLength=10):
    """Generate a random string of fixed length """
    return ''.join(random.sample(letters, stringLength))

print("===done===")

===done===


In [2]:
'''
print("Products between {} and {}".format(1, 75000000))
product_ids = [x for x in range(1, 75000000)]
dates = ['2020-07-01', '2020-07-02', '2020-07-03', '2020-07-04', '2020-07-05', '2020-07-06', '2020-07-07', '2020-07-08',
         '2020-07-09', '2020-07-10']
'''

'\nprint("Products between {} and {}".format(1, 75000000))\nproduct_ids = [x for x in range(1, 75000000)]\ndates = [\'2020-07-01\', \'2020-07-02\', \'2020-07-03\', \'2020-07-04\', \'2020-07-05\', \'2020-07-06\', \'2020-07-07\', \'2020-07-08\',\n         \'2020-07-09\', \'2020-07-10\']\n'

In [3]:
#   Generate sellers
seller_ids = [x for x in range(1, 10)]
sellers = [[0, "seller_0", 2500000]]
for s in tqdm(seller_ids):
    sellers.append([s, "seller_{}".format(s), random.randint(12000, 2000000)])
#   Save dataframe
df = pd.DataFrame(sellers)
df.columns = ["seller_id", "seller_name", "daily_target"]
df.to_csv("sellers.csv", index=False)


100%|████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<?, ?it/s]


In [4]:
#   Generate sales
'''
total_rows = 500000
prod_zero = int(total_rows * 0.95)
prod_others = total_rows - prod_zero + 1
df_array = [["order_id", "product_id", "seller_id", "date", "num_pieces_sold", "bill_raw_text"]]
with open('sales.csv', 'w', newline='') as f:
    csvwriter = csv.writer(f)
    csvwriter.writerows(df_array)'''

'#   Generate sales\ntotal_rows = 500000\nprod_zero = int(total_rows * 0.95)\nprod_others = total_rows - prod_zero + 1\ndf_array = [["order_id", "product_id", "seller_id", "date", "num_pieces_sold", "bill_raw_text"]]\nwith open(\'sales.csv\', \'w\', newline=\'\') as f:\n    csvwriter = csv.writer(f)\n    csvwriter.writerows(df_array)'

In [5]:
#   Generate products
'''
products = [[0, "product_0", 22]]
for p in tqdm(product_ids):
    products.append([p, "product_{}".format(p), random.randint(1, 150)])
#   Save dataframe
df = pd.DataFrame(products)
df.columns = ["product_id", "product_name", "price"]
df.to_csv("products.csv", index=False)
del df
del products

print("====done====")'''

'#   Generate products\nproducts = [[0, "product_0", 22]]\nfor p in tqdm(product_ids):\n    products.append([p, "product_{}".format(p), random.randint(1, 150)])\n#   Save dataframe\ndf = pd.DataFrame(products)\ndf.columns = ["product_id", "product_name", "price"]\ndf.to_csv("products.csv", index=False)\ndel df\ndel products\n\nprint("====done====")'

In [6]:
'''order_id = 0
for i in tqdm(range(0, 40)):
    df_array = []

    for i in range(0, prod_zero):
        order_id += 1
        df_array.append([order_id, 0, 0, random.choice(dates), random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

    df_array = []
    for i in range(0, prod_others):
        order_id += 1
        df_array.append(
            [order_id, random.choice(product_ids), random.choice(seller_ids), random.choice(dates),
             random.randint(1, 100), random_string(500)])

    with open('sales.csv', 'a', newline='') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerows(df_array)

print("Done")'''

'order_id = 0\nfor i in tqdm(range(0, 40)):\n    df_array = []\n\n    for i in range(0, prod_zero):\n        order_id += 1\n        df_array.append([order_id, 0, 0, random.choice(dates), random.randint(1, 100), random_string(500)])\n\n    with open(\'sales.csv\', \'a\', newline=\'\') as f:\n        csvwriter = csv.writer(f)\n        csvwriter.writerows(df_array)\n\n    df_array = []\n    for i in range(0, prod_others):\n        order_id += 1\n        df_array.append(\n            [order_id, random.choice(product_ids), random.choice(seller_ids), random.choice(dates),\n             random.randint(1, 100), random_string(500)])\n\n    with open(\'sales.csv\', \'a\', newline=\'\') as f:\n        csvwriter = csv.writer(f)\n        csvwriter.writerows(df_array)\n\nprint("Done")'

In [7]:
'''
spark = SparkSession.builder.master("local")\
.config("spark.sql.autoBroadcastJoinThreshold", -1)\
.appName("Exercise1").getOrCreate()

products = spark.read.csv("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products.csv", header=True, mode="DROPMALFORMED")

#products.show()

products.write.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products_parquet", mode="overwrite")

sales = spark.read.csv("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales.csv", header=True, mode="DROPMALFORMED")

#sales.show()

sales.repartition(200, col("product_id")).write.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales_parquet", mode="overwrite")

sellers = spark.read.csv("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers.csv", header=True, mode="DROPMALFORMED")

#sellers.show()

sellers.write.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers_parquet", mode="overwrite")

print("====done====")

'''

'\nspark = SparkSession.builder.master("local").config("spark.sql.autoBroadcastJoinThreshold", -1).appName("Exercise1").getOrCreate()\n\nproducts = spark.read.csv("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products.csv", header=True, mode="DROPMALFORMED")\n\n#products.show()\n\nproducts.write.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products_parquet", mode="overwrite")\n\nsales = spark.read.csv("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales.csv", header=True, mode="DROPMALFORMED")\n\n#sales.show()\n\nsales.repartition(200, col("product_id")).write.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales_parquet", mode="overwrite")\n\nsellers = spark.read.csv("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers.csv", header=True, mode="DROPMALFORMED")\n\n#sellers.show()\n\nsellers.write.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers_parquet", mode="overwrite")\n\nprint("====done====

# Find out how many orders, how many products and how many sellers are in the data.

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

#   Initialize the Spark session
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise1") \
    .getOrCreate()

#   Read the source tables in Parquet format
products_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products_parquet")
sales_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales_parquet")
sellers_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers_parquet")

#   Print the number of orders
print("Number of Orders: {}".format(sales_table.count()))

#   Print the number of sellers
print("Number of sellers: {}".format(sellers_table.count()))

#   Print the number of products
print("Number of products: {}".format(products_table.count()))

Number of Orders: 20000040
Number of sellers: 10
Number of products: 75000000


# How many products have been sold at least once? Which is the product contained in more orders?

In [9]:
#   Output how many products have been actually sold at least once
print("Number of products sold at least once")
sales_table.agg(countDistinct(col("product_id"))).show()

#   Output which is the product that has been sold in more orders
print("Product present in more orders")
sales_table.groupBy(col("product_id")).agg(count("*").alias("cnt")).orderBy(col("cnt").desc()).limit(1).show()

Number of products sold at least once
+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    993299|
+--------------------------+

Product present in more orders
+----------+--------+
|product_id|     cnt|
+----------+--------+
|         0|19000000|
+----------+--------+



In [10]:
sales_table.groupby(col("date")).agg(countDistinct(col("product_id")).alias("distinct_products_sold")).orderBy(
    col("distinct_products_sold").desc()).show()

+----------+----------------------+
|      date|distinct_products_sold|
+----------+----------------------+
|2020-07-04|                100294|
|2020-07-03|                100224|
|2020-07-10|                100218|
|2020-07-08|                100048|
|2020-07-05|                 99991|
|2020-07-06|                 99869|
|2020-07-09|                 99801|
|2020-07-02|                 99768|
|2020-07-01|                 99755|
|2020-07-07|                 99453|
+----------+----------------------+



In [11]:
#   What is the average revenue of the orders?
# Do the join and print the results
sales_table.join(products_table, sales_table["product_id"] == products_table["product_id"], "inner").agg(avg(products_table["price"] * sales_table["num_pieces_sold"])).show()

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|             1245.923923502153|
+------------------------------+



In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import IntegerType

# Create the Spark session
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "500mb") \
    .appName("Exercise1") \
    .getOrCreate()

# Read the source tables
products_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products_parquet")
sales_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales_parquet")
sellers_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers_parquet")

# Step 1 - Check and select the skewed keys 
# In this case we are retrieving the top 100 keys: these will be the only salted keys.
results = sales_table.groupby(sales_table["product_id"]).count().sort(col("count").desc()).limit(100).collect()

# Step 2 - What we want to do is:
#  a. Duplicate the entries that we have in the dimension table for the most common products, e.g.
#       product_0 will become: product_0-1, product_0-2, product_0-3 and so on
#  b. On the sales table, we are going to replace "product_0" with a random duplicate (e.g. some of them 
#     will be replaced with product_0-1, others with product_0-2, etc.)
# Using the new "salted" key will unskew the join

# Let's create a dataset to do the trick
REPLICATION_FACTOR = 101
l = []
replicated_products = []
for _r in results:
    replicated_products.append(_r["product_id"])
    for _rep in range(0, REPLICATION_FACTOR):
        l.append((_r["product_id"], _rep))
rdd = spark.sparkContext.parallelize(l)
replicated_df = rdd.map(lambda x: Row(product_id=x[0], replication=int(x[1])))
replicated_df = spark.createDataFrame(replicated_df)

#   Step 3: Generate the salted key
products_table = products_table.join(broadcast(replicated_df),
                                     products_table["product_id"] == replicated_df["product_id"], "left"). \
    withColumn("salted_join_key", when(replicated_df["replication"].isNull(), products_table["product_id"]).otherwise(
    concat(replicated_df["product_id"], lit("-"), replicated_df["replication"])))

sales_table = sales_table.withColumn("salted_join_key", when(sales_table["product_id"].isin(replicated_products),
                                                             concat(sales_table["product_id"], lit("-"),
                                                                    round(rand() * (REPLICATION_FACTOR - 1), 0).cast(
                                                                        IntegerType()))).otherwise(
    sales_table["product_id"]))

#   Step 4: Finally let's do the join
print(sales_table.join(products_table, sales_table["salted_join_key"] == products_table["salted_join_key"],
                       "inner").
      agg(avg(products_table["price"] * sales_table["num_pieces_sold"])).show())

print("Ok")

+------------------------------+
|avg((price * num_pieces_sold))|
+------------------------------+
|             1245.923923502153|
+------------------------------+

None
Ok


In [13]:
#   For each seller find the average % of the target amount brought by each order

#   Wrong way to do this - Skewed 
#   (Note that Spark will probably broadcast the table anyway, unless we forbid it throug the configuration paramters)
print(sales_table.join(sellers_table, sales_table["seller_id"] == sellers_table["seller_id"], "inner").withColumn(
    "ratio", sales_table["num_pieces_sold"]/sellers_table["daily_target"]
).groupBy(sales_table["seller_id"]).agg(avg("ratio")).show())

#   Correct way through broarcasting
print(sales_table.join(broadcast(sellers_table), sales_table["seller_id"] == sellers_table["seller_id"], "inner").withColumn(
    "ratio", sales_table["num_pieces_sold"]/sellers_table["daily_target"]
).groupBy(sales_table["seller_id"]).agg(avg("ratio")).show())

+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|        7|2.593509710759785...|
|        3|1.627171975063155...|
|        8|9.231037778437675E-5|
|        0|2.019736770553698E-5|
|        5|4.206816272236925...|
|        6|4.793659021438702E-5|
|        9|3.829272988800400...|
|        1|1.962610829274823E-4|
|        4|3.300641353602719...|
|        2|6.689119886281191E-5|
+---------+--------------------+

None
+---------+--------------------+
|seller_id|          avg(ratio)|
+---------+--------------------+
|        7|2.593509710759747...|
|        3|1.627171975063136...|
|        8|9.231037778437045E-5|
|        0|2.019736770525865...|
|        5|4.206816272236251E-5|
|        6| 4.79365902143776E-5|
|        9| 3.82927298880055E-5|
|        1|1.962610829274634...|
|        4|3.300641353603240...|
|        2|6.689119886277543E-5|
+---------+--------------------+

None


In [14]:
from pyspark.sql.window import Window
# Calcuate the number of pieces sold by each seller for each product
sales_table = sales_table.groupby(col("product_id"), col("seller_id")). \
    agg(sum("num_pieces_sold").alias("num_pieces_sold"))

# Create the window functions, one will sort ascending the other one descending. Partition by the product_id
# and sort by the pieces sold
window_desc = Window.partitionBy(col("product_id")).orderBy(col("num_pieces_sold").desc())
window_asc = Window.partitionBy(col("product_id")).orderBy(col("num_pieces_sold").asc())

# Create a Dense Rank (to avoid holes)
sales_table = sales_table.withColumn("rank_asc", dense_rank().over(window_asc)). \
    withColumn("rank_desc", dense_rank().over(window_desc))

# Get products that only have one row OR the products in which multiple sellers sold the same amount
# (i.e. all the employees that ever sold the product, sold the same exact amount)
single_seller = sales_table.where(col("rank_asc") == col("rank_desc")).select(
    col("product_id").alias("single_seller_product_id"), col("seller_id").alias("single_seller_seller_id"),
    lit("Only seller or multiple sellers with the same results").alias("type")
)

# Get the second top sellers
second_seller = sales_table.where(col("rank_desc") == 2).select(
    col("product_id").alias("second_seller_product_id"), col("seller_id").alias("second_seller_seller_id"),
    lit("Second top seller").alias("type")
)

# Get the least sellers and exclude those rows that are already included in the first piece
# We also exclude the "second top sellers" that are also "least sellers"
least_seller = sales_table.where(col("rank_asc") == 1).select(
    col("product_id"), col("seller_id"),
    lit("Least Seller").alias("type")
).join(single_seller, (sales_table["seller_id"] == single_seller["single_seller_seller_id"]) & (
        sales_table["product_id"] == single_seller["single_seller_product_id"]), "left_anti"). \
    join(second_seller, (sales_table["seller_id"] == second_seller["second_seller_seller_id"]) & (
        sales_table["product_id"] == second_seller["second_seller_product_id"]), "left_anti")

# Union all the pieces
union_table = least_seller.select(
    col("product_id"),
    col("seller_id"),
    col("type")
).union(second_seller.select(
    col("second_seller_product_id").alias("product_id"),
    col("second_seller_seller_id").alias("seller_id"),
    col("type")
)).union(single_seller.select(
    col("single_seller_product_id").alias("product_id"),
    col("single_seller_seller_id").alias("seller_id"),
    col("type")
))
union_table.show()

# Which are the second top seller and least seller of product 0?
union_table.where(col("product_id") == 0).show()

+----------+---------+------------+
|product_id|seller_id|        type|
+----------+---------+------------+
|  25078796|        4|Least Seller|
|  22113759|        7|Least Seller|
|   3384033|        1|Least Seller|
|  57452470|        2|Least Seller|
|  63583705|        8|Least Seller|
|  46764734|        5|Least Seller|
|  24992315|        5|Least Seller|
|   5342885|        7|Least Seller|
|  57356373|        6|Least Seller|
|  19581081|        1|Least Seller|
|  70135114|        4|Least Seller|
|  44179664|        7|Least Seller|
|  74608973|        7|Least Seller|
|  19657520|        8|Least Seller|
|  54810407|        7|Least Seller|
|  55049289|        6|Least Seller|
|  36289301|        6|Least Seller|
|   3147422|        1|Least Seller|
|  28365122|        9|Least Seller|
|  49196172|        4|Least Seller|
+----------+---------+------------+
only showing top 20 rows

+----------+---------+--------------------+
|product_id|seller_id|                type|
+----------+---------+

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Row, Window
from pyspark.sql.types import IntegerType
import hashlib

#   Init spark session
spark = SparkSession.builder \
    .master("local") \
    .config("spark.sql.autoBroadcastJoinThreshold", -1) \
    .config("spark.executor.memory", "1g") \
    .appName("Exercise1") \
    .getOrCreate()

#   Load source data
products_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\products_parquet")
sales_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sales_parquet")
sellers_table = spark.read.parquet("file:////C:\\Users\\dev30\\Documents\\pyspark_practice\\sellers_parquet")

#   Define the UDF function
def algo(order_id, bill_text):
    #   If number is even
    ret = bill_text.encode("utf-8")
    if int(order_id) % 2 == 0:
        #   Count number of 'A'
        cnt_A = bill_text.count("A")
        for _c in range(0, cnt_A):
            ret = hashlib.md5(ret).hexdigest().encode("utf-8")
        ret = ret.decode('utf-8')
    else:
        ret = hashlib.sha256(ret).hexdigest()
    return ret

#   Register the UDF function.
algo_udf = spark.udf.register("algo", algo)

#   Use the `algo_udf` to apply the aglorithm and then check if there is any duplicate hash in the table
sales_table.withColumn("hashed_bill", algo_udf(col("order_id"), col("bill_raw_text")))\
    .groupby(col("hashed_bill")).agg(count("*").alias("cnt")).where(col("cnt") > 1).show()

+-----------+---+
|hashed_bill|cnt|
+-----------+---+
+-----------+---+

